# 1、ConversationTokenMemory简介

举例1：

In [11]:
from langchain.memory import ConversationTokenBufferMemory
from langchain_community.chat_models import ChatTongyi
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

memory = ConversationTokenBufferMemory(
    llm=chat_model,
    max_token_limit=20
)

memory.save_context({"input":"你好吗？"},{"output":"我很好，谢谢！"})
memory.save_context({"input":"你是谁？"},{"output":"我是AI助手。"})

print(memory.load_memory_variables({}))

{'history': 'AI: 我是AI助手。'}


# 2、ConversationSummaryMemory的使用

举例1：

如果实例化ConversationSummaryMemory时没有历史消息，可以使用构造方法实例化

In [13]:
from langchain.memory import ConversationSummaryMemory
from langchain_community.chat_models import ChatTongyi
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

memory = ConversationSummaryMemory(llm=chat_model)

memory.save_context({"input":"你好吗？"},{"output":"我很好，谢谢！"})
memory.save_context({"input":"你是谁？"},{"output":"我是AI助手。"})
memory.save_context({"input":"你的生日是哪一天？"},{"output":"我不清楚。"})

print(memory.load_memory_variables({}))

{'history': 'The human asks how the AI is doing, and the AI responds that it is very well, thank you! The human then asks who the AI is, and the AI replies that it is an AI assistant. The human next asks the AI what its birthday is, and the AI responds that it doesn’t know.'}


举例2：如果实例化ConversationSummaryMemory时有历史消息，可以使用from_messages方法实例化

In [16]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_community.chat_models import ChatTongyi
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

history = ChatMessageHistory()
history.add_user_message("你好吗？")
history.add_ai_message("我很好，谢谢！")

memory = ConversationSummaryMemory.from_messages(llm=chat_model, chat_memory=history)

print(memory.load_memory_variables({}))

memory.save_context(inputs={"human":"我叫小明"}, outputs={"assistant":"你好呀"})

print(memory.load_memory_variables({}))

{'history': 'The human asks how the AI is doing, and the AI responds that it is very well, thank you!'}
{'history': 'The human asks how the AI is doing, and the AI responds that it is very well, thank you! The human then introduces themselves as Xiao Ming, and the AI greets them warmly with "你好呀".'}
